In [19]:
!pip install fuzzywuzzy



In [4]:
pip install levenshtein

Note: you may need to restart the kernel to use updated packages.


## Data Collection

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from fuzzywuzzy import process

In [6]:
url = "https://www.billboard.com/charts/hot-100/"

response = requests.get(url)
response.status_code
soup = BeautifulSoup(response.content)

In [7]:
songs = []


for song in soup.select('li h3.c-title'):
    songs.append(song.get_text(strip=True))
    


In [8]:
artist = []

for a in soup.select('li span.c-label.a-no-trucate'):
    artist.append(a.get_text(strip=True))



In [9]:
current_pos = []

for i in range(100):
    current_pos.append(i+1)

In [10]:
df_dict = {'current_pos': current_pos,
           'songs': songs,
           'artist': artist,
           }

billbord_df = pd.DataFrame(df_dict)

In [11]:
%store billbord_df

Stored 'billbord_df' (DataFrame)


In [12]:
billbord_df.head()

,current_pos,songs,artist
0,1,Lovin On Me,Jack Harlow
1,2,Lose Control,Teddy Swims
2,3,Beautiful Things,Benson Boone
3,4,Cruel Summer,Taylor Swift
4,5,Snooze,SZA


## User Interface

In [20]:
def song_searcher():   
    user_song = input('Enter song').title()

    if user_song in billbord_df['songs'].values:

        # Setup
        search = billbord_df['songs'] == user_song

        song = billbord_df.loc[search, 'songs'].iloc[0]
        artist = billbord_df.loc[search, 'artist'].iloc[0]
        rank = billbord_df.loc[search, 'current_pos'].iloc[0]

        while True:
            recomend_song = billbord_df['songs'].sample().iloc[0]
            recomend_artist = billbord_df['artist'].sample().iloc[0]
            if user_song != recomend_song:
                break

        # Output
        print(f'\nThe song you chose is in the Top 100 List\n{"*"*41}')
        print(f'Song: {song}\nArtist: {artist}\nRanked: {rank}\n{"*"*41}')
        print(f'\nSuggested as well:\n{recomend_song} by {recomend_artist}\n')

    else:
        print(f'\nThe song you are looking for is not in the list')

        # Fuzzy Matching 
        print(f'\nDid you maybe look for:')
        matches = process.extract(user_song, df['songs'], limit=2)
        for match in matches:
            print(match[0])
    


In [23]:
song_searcher()

Enter song Snooze



The song you chose is in the Top 100 List
*****************************************
Song: Snooze
Artist: SZA
Ranked: 5
*****************************************

Suggested as well:
FE!N by Chris Brown Featuring Davido & Lojay

